In [1]:
import xml.etree.ElementTree as ET
import os

def modify_bounding_box(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    for object_tag in root.findall('object'):
        bndbox = object_tag.find('bndbox')
        xmin = int(bndbox.find('xmin').text)
        ymin = int(bndbox.find('ymin').text)
        xmax = int(bndbox.find('xmax').text)
        ymax = int(bndbox.find('ymax').text)

        center_x = (xmin + xmax) // 2
        center_y = (ymin + ymax) // 2

        new_xmin = max(center_x - 50, 0)
        new_ymin = max(center_y - 50, 0)
        new_xmax = new_xmin + 100
        new_ymax = new_ymin + 100

        bndbox.find('xmin').text = str(new_xmin)
        bndbox.find('ymin').text = str(new_ymin)
        bndbox.find('xmax').text = str(new_xmax)
        bndbox.find('ymax').text = str(new_ymax)

    tree.write(xml_file)

def process_directory(directory):
    for filename in os.listdir(directory):
        if filename.endswith('.xml'):
            modify_bounding_box(os.path.join(directory, filename))

# Example usage
directories = ['test_images', 'train_images', 'valid_images']
for directory in directories:
    process_directory(directory)

In [1]:
import xml.etree.ElementTree as ET
import os

def convert_to_single_class(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    for object_tag in root.findall('object'):
        name_tag = object_tag.find('name')
        name_tag.text = "Cell"

    tree.write(xml_file)

def process_directory_for_single_class(directory):
    for filename in os.listdir(directory):
        if filename.endswith('.xml'):
            convert_to_single_class(os.path.join(directory, filename))

# Example usage
directories = ['test_images', 'train_images', 'valid_images']
for directory in directories:
    process_directory_for_single_class(directory)

In [2]:
import xml.etree.ElementTree as ET
import os

def remove_and_rename_classes(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    objects = root.findall('object')
    for object_tag in objects:
        class_name = object_tag.find('name').text
        if class_name == "Negative for intraepithelial lesion":
            root.remove(object_tag)
        else:
            object_tag.find('name').text = "Anomaly"

    tree.write(xml_file)

def process_directory_for_removal_and_rename(directory):
    for filename in os.listdir(directory):
        if filename.endswith('.xml'):
            remove_and_rename_classes(os.path.join(directory, filename))

# Example usage
directories = ['test_images', 'train_images', 'valid_images']
for directory in directories:
    process_directory_for_removal_and_rename(directory)

In [2]:
import xml.etree.ElementTree as ET
import os
from PIL import Image

def create_directories(base_dir, sub_dirs):
    if not os.path.exists(base_dir):
        os.mkdir(base_dir)
    
    for dir in sub_dirs:
        path = os.path.join(base_dir, dir)
        if not os.path.exists(path):
            os.mkdir(path)

def crop_and_save(image_path, box, save_path):
    image = Image.open(image_path)
    width, height = image.size

    xmin, ymin, xmax, ymax = box
    center_x, center_y = (xmin + xmax) // 2, (ymin + ymax) // 2

    # Ensure the crop stays within the image boundaries
    crop_xmin = max(center_x - 50, 0)
    crop_ymin = max(center_y - 50, 0)
    crop_xmax = min(center_x + 50, width)
    crop_ymax = min(center_y + 50, height)

    cropped_image = image.crop((crop_xmin, crop_ymin, crop_xmax, crop_ymax))
    cropped_image.save(save_path)

def process_xml(xml_file, img_dir, crop_dir):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    filename = root.find('filename').text
    image_path = os.path.join(img_dir, filename)

    # Counter for unique crop names
    counter = 1

    for object_tag in root.findall('object'):
        class_name = object_tag.find('name').text
        bndbox = object_tag.find('bndbox')
        box = (
            int(bndbox.find('xmin').text),
            int(bndbox.find('ymin').text),
            int(bndbox.find('xmax').text),
            int(bndbox.find('ymax').text)
        )

        # Create class directory if not exists
        class_dir = os.path.join(crop_dir, class_name)
        create_directories(class_dir, [])

        # Unique filename for each crop
        crop_save_path = os.path.join(class_dir, f"{os.path.splitext(filename)[0]}_{class_name}_{counter}.png")
        crop_and_save(image_path, box, crop_save_path)

        counter += 1

def main():
    dataset_splits = ['train']
    base_crop_dir = 'crops'

    for split in dataset_splits:
        img_dir = os.path.join(split + '_images')
        xml_dir = img_dir  # Assuming XML files are in the same directory as images
        crop_dir = os.path.join(base_crop_dir, split)

        create_directories(base_crop_dir, [split])

        for xml_file in os.listdir(xml_dir):
            if xml_file.endswith('.xml'):
                process_xml(os.path.join(xml_dir, xml_file), img_dir, crop_dir)

if __name__ == '__main__':
    main()